<a href="https://colab.research.google.com/github/Maheenms/GoogleCoLab/blob/main/unreal_restaurant_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [25]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [26]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [27]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_2/unreal_restaurant_tweets.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("unreal_restaurant_tweets.csv"), sep=",", header=True)

# Show DataFrame
df.show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------+
|Restaurant Tweets                                                                                                     |
+----------------------------------------------------------------------------------------------------------------------+
|@UnrealRestaurant service was super slow today. Pick up your game, or I'm not going back.                             |
|@UnrealRestaurant Not so sure about the underwater restaurant experience. Kinda felt like I was drowning.             |
|@UnrealRestaurant when I'm away, you're all I think about. You make me feel like a natural mermaid.                   |
|@UnrealRestaurant Are your opening hours listed online accurate? You were closed at 4pm Monday.                       |
|"@UnrealRestaurant The ""pineapple-under-the-sea"" menu item makes me feel bad for Spongebob. Don't eat his house. :("|
+-------------------------------

In [28]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol = "Restaurant Tweets", outputCol = "Word List")
df= tokenizer.transform(df)
df.show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|Restaurant Tweets                                                                                                     |Word List                                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|@UnrealRestaurant service was super slow today. Pick up your game, or I'm not going back.                             |[@unrealrestaurant, service, was, super, slow, today., pick, up, your, game,, or, i'm, not, going, back.]    

In [29]:
# Remove stop words
  # use the custom stop word remover - overrides the default stop word remover
  # custom stop word remover only removes listed words

# list of custom stop words
stopWordsList = ["@UnrealRestaurant", "@unrealrestaurant", "\"@unrealrestaurant"]

# use the custom stop word remover
stopRemover = StopWordsRemover(inputCol="Word List", outputCol="Filtered", stopWords=stopWordsList)

# apply the custom stop word remover
df = stopRemover.transform(df)

# use the default stop word remover
stopRemover2 = StopWordsRemover(inputCol="Filtered", outputCol="More Filtered")

# apply the default stop word remover
df = stopRemover2.transform(df)

df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+
|Restaurant Tweets                                                                                                     |Word List                                                                                                                              |Filtered                                                                                                           |More Filtered                                                                            |
+-------------------------------------------------------------------------

In [30]:
# Run the hashing term frequency
hash = HashingTF(inputCol='More Filtered', outputCol="Hashing Frequencies") # use the default number of features

# Transform into a DF
df = hash.transform(df)
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Restaurant Tweets                                                                                                     |Word List                                                                                                                              |Filtered                                                                                                           |More Filtered                                

In [31]:
# Fit the IDF on the data set 

# Fit the IDF on the data set 
idf = IDF(inputCol='Hashing Frequencies', outputCol="IDF Frequencies") # use the default number of features
idfModel = idf.fit(df)
df = idfModel.transform(df)
# Display the DataFrame
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Restaurant Tweets                                                                                                     |Word List                  

In [ ]:
# Display the DataFrame